In [13]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine, text
from sqlalchemy.types import Integer, BigInteger, String, Date, Float, DateTime
import urllib
import os

print("--- ETL AVANÇADO: CARGA TIPADA (PYTHON -> SQL SERVER) ---")

# --- 1. CONFIGURAÇÕES ---
SERVER = 'DESKTOP-IVTRK7E'
DATABASE = 'MC_Supermercados'
DRIVER = 'ODBC Driver 17 for SQL Server'

# String de Conexão
params = urllib.parse.quote_plus(
    f"DRIVER={{{DRIVER}}};SERVER={SERVER};DATABASE={DATABASE};Trusted_Connection=yes;"
)
engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}", fast_executemany=True)

--- ETL AVANÇADO: CARGA TIPADA (PYTHON -> SQL SERVER) ---


In [14]:
# --- 2. DEFINIÇÃO EXPLÍCITA DOS TIPOS ---
esquema_tipos = {
    'fato_vendas': {
        'id_venda': BigInteger(),
        'id_loja': Integer(),
        'id_produto': BigInteger(),
        'id_cliente': BigInteger(),
        'id_funcionario': Integer(),
        'id_caixa': Integer(),
        'id_data': Integer(),
        'data_venda': Date(),
        'quantidade': Integer(),
        'valor_unitario_tabela': Float(precision=2),
        'desconto_promo_unit': Float(precision=2),
        'desconto_clube_unit': Float(precision=2),
        'valor_final_total': Float(precision=2),
        'id_promocao': Integer()
    },
    'dim_clientes': { 'id_cliente': BigInteger(), 'nome': String(100), 'idade': Integer(), 'id_data_cadastro': Integer() },
    'dim_produtos': { 
        'sku': BigInteger(), 
        'custo_unitario': Float(precision=2), 
        'preco_venda_sup': Float(precision=2), 
        'id_fornecedor': Integer()
    },
    'dim_lojas': { 'id_loja': Integer() },
    
    # GARANTINDO TIPAGEM DE FORNECEDORES
    'dim_fornecedores': {
        'id_fornecedor': Integer(),
        'nome': String(100),
        'cidade': String(50)
    },
    
    # ADICIONADO AGORA: GARANTINDO TIPAGEM DE FUNCIONÁRIOS
    'dim_funcionarios': {
        'id_funcionario': Integer(),
        'id_loja': Integer(),
        'nome': String(100),
        'cargo': String(50),
        'departamento': String(50)
    }
}

In [15]:
# --- 3. EXECUÇÃO DO ETL ---
arquivos = [
    ('dim_lojas.csv', 'dim_lojas'),
    ('dim_fornecedores.csv', 'dim_fornecedores'),
    ('dim_funcionarios.csv', 'dim_funcionarios'),
    ('dim_clientes.csv', 'dim_clientes'),
    ('dim_produtos.csv', 'dim_produtos'),
    ('dim_promocoes.csv', 'dim_promocoes'),
    ('dim_datas.csv', 'dim_datas'),
    ('fato_vendas.csv', 'fato_vendas')
]

for arquivo, tabela in arquivos:
    if os.path.exists(arquivo):
        print(f"Processando {tabela}...")
        df = pd.read_csv(arquivo)
        
        # TRATAMENTO PRÉVIO DE DATAS (CORRIGIDO)
        for col in df.columns:
            # Se for ID (chave), não mexa!
            if col.startswith('id_'):
                continue
                
            # Se for data real (data_venda, data_inicio, etc), converte
            if 'data' in col or 'inicio' in col or 'fim' in col:
                try:
                    df[col] = pd.to_datetime(df[col])
                except: pass
        
        # TRATAMENTO DE NULOS
        tipos_sql = esquema_tipos.get(tabela)
        
        df.to_sql(
            tabela, 
            engine, 
            if_exists='replace',
            index=False, 
            chunksize=5000,
            dtype=tipos_sql
        )
        print(f"✅ {tabela} carregada com tipagem correta!")
    else:
        print(f"⚠️ Arquivo {arquivo} não encontrado na pasta!")


Processando dim_lojas...
✅ dim_lojas carregada com tipagem correta!
Processando dim_fornecedores...
✅ dim_fornecedores carregada com tipagem correta!
Processando dim_funcionarios...
✅ dim_funcionarios carregada com tipagem correta!
Processando dim_clientes...
✅ dim_clientes carregada com tipagem correta!
Processando dim_produtos...
✅ dim_produtos carregada com tipagem correta!
Processando dim_promocoes...
✅ dim_promocoes carregada com tipagem correta!
Processando dim_datas...
✅ dim_datas carregada com tipagem correta!
Processando fato_vendas...
✅ fato_vendas carregada com tipagem correta!


In [16]:
# --- 4. APLICAÇÃO DE CHAVES E ÍNDICES (SQL SERVER) ---
print("\n--- APLICANDO CHAVES E ÍNDICES (PERFORMANCE) ---")

comandos_sql = [
    # 1. Dimensão Lojas
    "ALTER TABLE dim_lojas ALTER COLUMN id_loja INT NOT NULL",
    "ALTER TABLE dim_lojas ADD CONSTRAINT PK_dim_lojas PRIMARY KEY (id_loja)",

    # 2. Dimensão Produtos
    "ALTER TABLE dim_produtos ALTER COLUMN sku BIGINT NOT NULL",
    "ALTER TABLE dim_produtos ADD CONSTRAINT PK_dim_produtos PRIMARY KEY (sku)",

    # 3. Dimensão Clientes
    "ALTER TABLE dim_clientes ALTER COLUMN id_cliente BIGINT NOT NULL",
    "ALTER TABLE dim_clientes ADD CONSTRAINT PK_dim_clientes PRIMARY KEY (id_cliente)",
    
    # 4. Dimensão Fornecedores (Adicionamos recentemente)
    "ALTER TABLE dim_fornecedores ALTER COLUMN id_fornecedor INT NOT NULL",
    "ALTER TABLE dim_fornecedores ADD CONSTRAINT PK_dim_fornecedores PRIMARY KEY (id_fornecedor)",

    # 5. Dimensão Funcionários (Boas práticas: vamos garantir a chave dela também)
    "ALTER TABLE dim_funcionarios ALTER COLUMN id_funcionario INT NOT NULL",
    "ALTER TABLE dim_funcionarios ADD CONSTRAINT PK_dim_funcionarios PRIMARY KEY (id_funcionario)",

    # 6. Fato Vendas - Ajuste de Colunas Obrigatórias (NOT NULL)
    # IDs antigos
    "ALTER TABLE fato_vendas ALTER COLUMN id_venda BIGINT NOT NULL",
    "ALTER TABLE fato_vendas ALTER COLUMN id_loja INT NOT NULL",
    "ALTER TABLE fato_vendas ALTER COLUMN id_produto BIGINT NOT NULL",
    "ALTER TABLE fato_vendas ALTER COLUMN id_data INT NOT NULL",
    # IDs novos (Operador e Caixa)
    "ALTER TABLE fato_vendas ALTER COLUMN id_funcionario INT NOT NULL",
    "ALTER TABLE fato_vendas ALTER COLUMN id_caixa INT NOT NULL",
    
    # 7. Fato Vendas - Criação de Índices
    # Índice Principal (Clustered) - Organiza a tabela fisicamente
    "CREATE CLUSTERED INDEX CX_FatoVendas ON fato_vendas(id_venda)",
    
    # Índices Auxiliares (Para os filtros do Power BI ficarem rápidos)
    "CREATE INDEX idx_fato_loja ON fato_vendas(id_loja)",
    "CREATE INDEX idx_fato_data ON fato_vendas(data_venda)",
    "CREATE INDEX idx_fato_cliente ON fato_vendas(id_cliente)",
    "CREATE INDEX idx_fato_func ON fato_vendas(id_funcionario)"
]

with engine.connect() as conn:
    for sql in comandos_sql:
        try:
            conn.execute(text(sql))
            conn.commit()
        except Exception as e:
            print(f"Aviso SQL: {e}")

print("\n🎉 PROCESSO CONCLUÍDO! Banco 100% tipado e indexado.")


--- APLICANDO CHAVES E ÍNDICES (PERFORMANCE) ---

🎉 PROCESSO CONCLUÍDO! Banco 100% tipado e indexado.
